In [49]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from extract import extract_covid_data
from transformation import generate_covid_info
from load import load_data
from datetime import datetime
import argparse
from pyspark.sql.functions import map_concat, lit, create_map, explode, col
from pyspark.sql.types import DoubleType
from pyspark.sql import Row

In [2]:
def get_spark_utils():
    """
    !!!DO NOT TOUCH!!!
    This function returns spark context object and spark session object.
    These are the entry point into all functionality in Spark.
    :return: SparkContext, SparkSession
    """
    conf = SparkConf().setAppName("Covid"). \
        set("spark.mongodb.input.uri", "mongodb://127.0.0.1"). \
        set("spark.mongodb.output.uri", "mongodb://127.0.0.1"). \
        set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1"). \
        set("spark.sql.debug.maxToStringFields", 1000)
    spark = SparkSession.builder.master("local[*]").config(conf=conf).getOrCreate()
    sc = spark.sparkContext
    return sc, spark

In [3]:
sc, spark = get_spark_utils()

22/12/26 20:28:38 WARN Utils: Your hostname, Uzays-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 161.9.123.126 instead (on interface en0)
22/12/26 20:28:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/lib/python3.10/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/uzay/.ivy2/cache
The jars for the packages stored in: /Users/uzay/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b725e627-bee3-41d4-b8db-510dcc21c532;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 204ms :: artifacts dl 16ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts  

In [4]:
raw_data = extract_covid_data()

In [5]:
raw_data_df = spark.createDataFrame(raw_data)

In [6]:
covid_info = generate_covid_info(raw_data_df)

In [9]:
type(covid_info)

NoneType

In [10]:
raw_data_df_exploded =  raw_data_df.select(raw_data_df.location, explode(raw_data_df.data))

In [11]:
raw_data_df_exploded = raw_data_df_exploded.withColumn("location_map", create_map(
        lit("location"), col("location"))).drop("location")

In [12]:
raw_data_df_exploded.show(10)

22/12/26 20:29:46 WARN TaskSetManager: Stage 0 contains a task of very large size (17954 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+
|                 col|        location_map|
+--------------------+--------------------+
|{date -> 2020-02-...|{location -> Afgh...|
|{date -> 2020-02-...|{location -> Afgh...|
|{date -> 2020-02-...|{location -> Afgh...|
|{date -> 2020-02-...|{location -> Afgh...|
|{date -> 2020-02-...|{location -> Afgh...|
|{date -> 2020-02-...|{location -> Afgh...|
|{date -> 2020-03-...|{location -> Afgh...|
|{date -> 2020-03-...|{location -> Afgh...|
|{date -> 2020-03-...|{location -> Afgh...|
|{date -> 2020-03-...|{location -> Afgh...|
+--------------------+--------------------+
only showing top 10 rows



In [13]:
data_map = raw_data_df_exploded.select(map_concat("col", "location_map").alias("data_map"))

In [14]:
data_map.show(10)

22/12/26 20:29:51 WARN TaskSetManager: Stage 1 contains a task of very large size (17954 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+
|            data_map|
+--------------------+
|{date -> 2020-02-...|
|{date -> 2020-02-...|
|{date -> 2020-02-...|
|{date -> 2020-02-...|
|{date -> 2020-02-...|
|{date -> 2020-02-...|
|{date -> 2020-03-...|
|{date -> 2020-03-...|
|{date -> 2020-03-...|
|{date -> 2020-03-...|
+--------------------+
only showing top 10 rows



In [15]:
dml = data_map.select('data_map').rdd.flatMap(lambda x: x).collect()

22/12/26 20:29:54 WARN TaskSetManager: Stage 2 contains a task of very large size (17954 KiB). The maximum recommended task size is 1000 KiB.


In [16]:
df = spark.read.json(sc.parallelize(dml)).na.fill('')

22/12/26 20:30:08 WARN TaskSetManager: Stage 3 contains a task of very large size (15983 KiB). The maximum recommended task size is 1000 KiB.


In [18]:
type(df)

pyspark.sql.dataframe.DataFrame

In [53]:
covid_info = generate_covid_info(raw_data_df)

In [21]:
df

DataFrame[date: string, excess_mortality: string, excess_mortality_cumulative: string, excess_mortality_cumulative_absolute: string, excess_mortality_cumulative_per_million: string, hosp_patients: string, hosp_patients_per_million: string, icu_patients: string, icu_patients_per_million: string, location: string, new_cases: string, new_cases_per_million: string, new_cases_smoothed: string, new_cases_smoothed_per_million: string, new_deaths: string, new_deaths_per_million: string, new_deaths_smoothed: string, new_deaths_smoothed_per_million: string, new_people_vaccinated_smoothed: string, new_people_vaccinated_smoothed_per_hundred: string, new_tests: string, new_tests_per_thousand: string, new_tests_smoothed: string, new_tests_smoothed_per_thousand: string, new_vaccinations: string, new_vaccinations_smoothed: string, new_vaccinations_smoothed_per_million: string, people_fully_vaccinated: string, people_fully_vaccinated_per_hundred: string, people_vaccinated: string, people_vaccinated_per

In [52]:
type(covid_info)

NoneType